In [2]:
!uv pip install -e ../.

Using Python 3.11.14 environment at: /Users/ariff.a/Documents/GitHub/StreamingDataset/.venv
Resolved 22 packages in 1.38s                                        
   Building chinidataset @ file:///Users/ariff.a/Documents/GitHub/StreamingDatas
   Building chinidataset @ file:///Users/ariff.a/Documents/GitHub/StreamingDatas
   Building chinidataset @ file:///Users/ariff.a/Documents/GitHub/StreamingDatas
      Built chinidataset @ file:///Users/ariff.a/Documents/GitHub/StreamingDatas
Prepared 1 package in 592ms                                              
Uninstalled 1 package in 0.94ms
Installed 1 package in 5ms (from file:///Users/ariff.a/Docum
 - chinidataset==0.2.1 (from file:///Users/ariff.a/Documents/GitHub/StreamingDataset)
 + chinidataset==0.2.0 (from file:///Users/ariff.a/Documents/GitHub/StreamingDataset)


In [3]:
from datasets import load_dataset

ds = load_dataset(
    "parquet",
    data_files="hf://datasets/wikimedia/wikipedia/20231101.en/train-00000-of-00041.parquet",
    split="train",
)
print(f"Loaded {len(ds):,} articles")

/Users/ariff.a/Documents/GitHub/StreamingDataset/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loaded 156,289 articles


In [4]:
from collections import Counter

counter = Counter()
for row in ds:
    counter.update(row["text"].split())

vocab = {"<unk>": 0}
for i, (word, _) in enumerate(counter.most_common(49_999), start=1):
    vocab[word] = i

print(f"Vocab size: {len(vocab):,}")

Vocab size: 50,000


In [6]:
import numpy as np

UNK_ID = 0

def transform(row):
    ids = [vocab.get(w, UNK_ID) for w in row["text"].split()]
    return {"input_ids": np.array(ids, dtype=np.uint32)}

In [10]:
from chinidataset import ParquetWriter
from tqdm import tqdm

In [11]:
%%time

columns = {"input_ids": "uint32[]"}

with ParquetWriter(out=OUT, columns=columns, exist_ok=True) as writer:
    for row in tqdm(ds, desc="Writing"):
        writer.write(transform(row))

Directory /Users/ariff.a/Documents/GitHub/StreamingDataset/examples/wiki_tokenized exists; removing contents.
Writing: 100%|██████████| 156289/156289 [00:14<00:00, 10821.04it/s]


CPU times: user 13.1 s, sys: 597 ms, total: 13.7 s
Wall time: 14.5 s


In [3]:
import boto3
import os

s3 = boto3.client('s3', region_name='ap-southeast-1')

bucket = 'ariffnazhan-chinidataset-s3-remote-testing'
local_folder = './wiki_tokenized'
s3_prefix = 'wikipedia-shard-0'  # s3 path stays the same

for root, dirs, files in os.walk(local_folder):
    for filename in files:
        local_path = os.path.join(root, filename)
        # build the S3 key: wikipedia-shard-0/index.json, wikipedia-shard-0/shard.00000.parquet, etc.
        relative_path = os.path.relpath(local_path, local_folder)
        s3_key = f'{s3_prefix}/{relative_path}'

        print(f'Uploading {local_path} -> s3://{bucket}/{s3_key}')
        s3.upload_file(local_path, bucket, s3_key)

print('Done!')

Uploading ./wiki_tokenized/shard.00003.parquet -> s3://ariffnazhan-chinidataset-s3-remote-testing/wikipedia-shard-0/shard.00003.parquet
Uploading ./wiki_tokenized/shard.00002.parquet -> s3://ariffnazhan-chinidataset-s3-remote-testing/wikipedia-shard-0/shard.00002.parquet
Uploading ./wiki_tokenized/shard.00000.parquet -> s3://ariffnazhan-chinidataset-s3-remote-testing/wikipedia-shard-0/shard.00000.parquet
Uploading ./wiki_tokenized/shard.00001.parquet -> s3://ariffnazhan-chinidataset-s3-remote-testing/wikipedia-shard-0/shard.00001.parquet
Uploading ./wiki_tokenized/shard.00004.parquet -> s3://ariffnazhan-chinidataset-s3-remote-testing/wikipedia-shard-0/shard.00004.parquet
Uploading ./wiki_tokenized/shard.00005.parquet -> s3://ariffnazhan-chinidataset-s3-remote-testing/wikipedia-shard-0/shard.00005.parquet
Uploading ./wiki_tokenized/index.json -> s3://ariffnazhan-chinidataset-s3-remote-testing/wikipedia-shard-0/index.json
Uploading ./wiki_tokenized/shard.00006.parquet -> s3://ariffnazhan

In [13]:
!uv pip install "botocore[crt]"

Using Python 3.11.14 environment at: /Users/ariff.a/Documents/GitHub/StreamingDataset/.venv
Resolved 6 packages in 729ms                                         
⠙ Preparing packages... (0/1)                                                   
⠙ Preparing packages... (0/1)-------------------     0 B/3.25 MiB            
⠙ Preparing packages... (0/1)------------------- 16.00 KiB/3.25 MiB          
⠙ Preparing packages... (0/1)------------------- 32.00 KiB/3.25 MiB          
⠙ Preparing packages... (0/1)------------------- 48.00 KiB/3.25 MiB          
⠙ Preparing packages... (0/1)------------------- 60.99 KiB/3.25 MiB          
⠙ Preparing packages... (0/1)------------------- 76.99 KiB/3.25 MiB          
⠙ Preparing packages... (0/1)------------------- 92.99 KiB/3.25 MiB          
⠙ Preparing packages... (0/1)------------------- 108.99 KiB/3.25 MiB         
⠙ Preparing packages... (0/1)------------------- 124.99 KiB/3.25 MiB         
⠙ Preparing packages... (0/1)------------------- 140.99

In [4]:
from chinidataset import StreamingDataset

ds = StreamingDataset(
    local="/tmp/s3_cache",
    remote="s3://ariffnazhan-chinidataset-s3-remote-testing/wikipedia-shard-0",
)

print(f"Total samples: {len(ds):,}")
print(f"First sample keys: {list(ds[0].keys())}")
print(f"First sample input_ids shape: {ds[0]['input_ids'].shape}")

for i, sample in enumerate(ds):
    if i >= 5:
        break
    print(f"Sample {i}: input_ids length = {len(sample['input_ids'])}")

Total samples: 156,289
First sample keys: ['input_ids']
First sample input_ids shape: (6712,)
Sample 0: input_ids length = 6712
Sample 1: input_ids length = 3875
Sample 2: input_ids length = 1838
Sample 3: input_ids length = 12052
Sample 4: input_ids length = 7616
